In [16]:
import torch
import lfcm
from collections import OrderedDict
import numpy as np
import customTransform
from dotenv import load_dotenv
import os

load_dotenv()

checkpoint_file = "checkpoints/fcm_e4b16.pth"

In [17]:
def image_to_tensor(filepath):
    image = np.zeros((3, 299, 299), dtype=np.float32)
    try:
        image = customTransform.filepath_to_image(filepath)
        image = customTransform.rescale(image, 299)
        image = customTransform.preprocess_image_to_np_arr(image)
        image = torch.from_numpy(image.copy())
    except:
        image = np.zeros((3, 299, 299), dtype=np.float32)
        image = torch.from_numpy(image.copy())

    return image

def text_to_tensor(id):
    text = np.zeros(150, dtype=np.float32)
    try:
        filepath = os.getenv('EMBED_TT_PATH')
        for i,line in enumerate(open(filepath)):
            data = line.strip().split(',')
            tweet_id = data[0]

            if id == tweet_id:
                arr = np.array(list(map(float, data[1:])))
                text = arr
    except Exception as e:
        text = np.zeros(150, dtype=np.float32)

    return text


def image_text_to_tensor(id):
    image_text = np.zeros(150, dtype=np.float32)
    try:
        filepath = os.getenv('EMBED_IT_PATH')
        for i,line in enumerate(open(filepath)):
            data = line.strip().split(',')
            tweet_id = data[0]

            if id == tweet_id:
                arr = np.array(list(map(float, data[1:])))
                image_text = arr
    except Exception as e:
        print('error:',e)
        image_text = np.zeros(150, dtype=np.float32)

    return image_text.copy()


def get_input_tensors(input_data):
    image = np.zeros((3, 299, 299), dtype=np.float32)
    text = np.zeros(150, dtype=np.float32)  # hidden state dimension of lstm
    image_text = np.zeros(150, dtype=np.float32)
    comments = np.zeros(150, dtype=np.float32)

    if "image_url" in input_data:
        image = image_to_tensor(input_data["image_url"])
    else:
        image = torch.from_numpy(image.copy())
    
    if 'tweet_id' in input_data:
        text = text_to_tensor(input_data['tweet_id'])
    else:
        text = torch.from_numpy(text.copy())
    
    # text = text.astype(np.float32)

    if 'tweet_id' in input_data:
        image_text = image_text_to_tensor(input_data['tweet_id'])
    else:
        image_text = image_text.copy()

    # image_text = image_text.astype(np.float32)

    comments = torch.from_numpy(comments.copy())
    text = torch.from_numpy(text.copy())
    image_text = torch.from_numpy(image_text.copy())

    return {
        "image": image,
        "image_text": image_text,
        "text": text,
        "comments": comments,
    }


def load_model():
    model = lfcm.OldModel(gpu=0)
    checkpoint = torch.load(checkpoint_file)
    model = lfcm.OldModel()
    model = torch.nn.DataParallel(model, device_ids=[0]).cuda(0)
    model.load_state_dict(checkpoint)
    return model


def predict(model, input_tensors):
    in_ten = input_tensors
    image = in_ten["image"]
    image_text = in_ten["image_text"]
    text = in_ten["text"]

    with torch.no_grad():
        model.eval()
        output = model(image.unsqueeze(0), image_text.unsqueeze(0), text.unsqueeze(0))

    return output

In [2]:
# input_tensors = get_input_tensors(dict())

In [6]:
input_data = {
    "tweet_text": "Is she daft BC has had a carbon tax in place since July 2008 The PM wasnt even in politics when it was initiated THERE IS NO TRUDEAU CARBON TAX in BC YOU MORON Citizens of Kelowna WERE ALL thinking of you with a crisis in your city amp a village idiot as MP Stay safe ",
    "image_url": "",
    "comments": [
        "Yes she is shamelessly daft",
        "Seems you need to be really stunned to qualify for that conservative ideology treatment",
        "Facts dont matter to these",
        "Shes a feckless kunt along with her cult climate deniers",
        "And it has worked marvelously",
        "CPC MPs do not care if they make sense  it is all about pushing misinformation to discredit the PM",
        "two thumbs up when they are all gone please post sid by shots to let us know please",
        "YAAAAWWWWNNN Youre as phoney as Trudeaus carbon taxDildo",
        "I thought journalists always fact checked their stories Ooops",
        "Daft is kind for this type of dangerous misinformation",
    ],
    "tweet_id": "1693318988380442696",
}
model = load_model()
input_tensors = get_input_tensors(input_data)
prediction = predict(model, input_tensors)
print(prediction)

Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers
Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers
tensor([[-0.0173, -0.1286]], device='cuda:0')


In [7]:
input_data = {
    "tweet_text": "Is she daft BC has had a carbon tax in place since July 2008 The PM wasnt even in politics when it was initiated THERE IS NO TRUDEAU CARBON TAX in BC YOU MORON Citizens of Kelowna WERE ALL thinking of you with a crisis in your city amp a village idiot as MP Stay safe ",
    "image_url": "",
    "comments": [
        "Yes she is shamelessly daft",
        "Seems you need to be really stunned to qualify for that conservative ideology treatment",
        "Facts dont matter to these",
        "Shes a feckless kunt along with her cult climate deniers",
        "And it has worked marvelously",
        "CPC MPs do not care if they make sense  it is all about pushing misinformation to discredit the PM",
        "two thumbs up when they are all gone please post sid by shots to let us know please",
        "YAAAAWWWWNNN Youre as phoney as Trudeaus carbon taxDildo",
        "I thought journalists always fact checked their stories Ooops",
        "Daft is kind for this type of dangerous misinformation",
    ],
    "tweet_id": "1693318988380442696",
}
input_tensors = get_input_tensors(input_data)


In [8]:
input_tensors['image']

tensor([[[ 150.9930,  150.9930,  150.9930,  ...,  147.9930,  144.9930,
           144.9930],
         [ 150.9930,  150.9930,  150.9930,  ...,  147.9930,  145.9930,
           145.9930],
         [ 150.9930,  150.9930,  150.9930,  ...,  150.9930,  147.9930,
           147.9930],
         ...,
         [ 150.9930,  150.9930,  150.9930,  ...,  -93.0070,  -63.0070,
             5.9930],
         [ 150.9930,  150.9930,  150.9930,  ...,   15.9930,  101.9930,
           146.9930],
         [ 150.9930,  150.9930,  150.9930,  ...,  131.9930,  135.9930,
           128.9930]],

        [[ 138.3312,  138.3312,  138.3312,  ...,  138.3312,  138.3312,
           138.3312],
         [ 138.3312,  138.3312,  138.3312,  ...,  138.3312,  138.3312,
           138.3312],
         [ 138.3312,  138.3312,  138.3312,  ...,  137.3312,  135.3312,
           135.3312],
         ...,
         [ 138.3312,  138.3312,  138.3312,  ...,  -96.6688,  -70.6688,
            -1.6688],
         [ 138.3312,  138.3312,  138.331

In [9]:
input_tensors['text']

tensor([ 7.3665e-03,  9.8336e-03,  6.2512e-02,  4.0015e-02, -2.1601e-03,
         2.4825e-02, -3.9657e-02,  2.7556e-02,  1.0487e-01, -3.4718e-03,
        -7.9896e-02, -1.0934e-02,  1.0123e-01,  2.9717e-01,  2.2180e-01,
         1.2709e-01, -8.4463e-03,  5.1178e-01, -4.6893e-01, -2.8085e-01,
         4.0842e-03,  4.9391e-02,  3.0191e-02, -1.6089e-01, -7.4724e-03,
         3.3243e-02, -2.8608e-02, -3.9465e-03, -8.9247e-03, -7.2318e-02,
         2.5304e-02, -4.1184e-04, -8.8525e-03, -4.2712e-02,  2.9370e-02,
         3.2649e-01,  2.2707e-02, -1.0853e-02,  8.5953e-03,  1.5991e-01,
         4.9769e-02,  7.9289e-03, -4.2235e-02, -1.3066e-02, -9.6779e-03,
        -1.6154e-02,  2.2624e-02,  2.8407e-01, -1.4514e-01,  4.2968e-02,
        -7.2501e-02,  3.7755e-02,  3.6654e-01, -3.0091e-02, -7.2391e-03,
         1.6353e-01,  2.7134e-02, -5.0871e-02,  6.5337e-02,  4.8362e-03,
         1.2670e-01, -4.1465e-03, -5.4035e-02, -3.9766e-03, -4.5078e-02,
         3.5893e-01, -1.3961e-01, -6.8567e-02, -5.3

In [10]:
input_tensors['image_text']

tensor([-0.0349, -0.3688, -0.2289, -0.2531, -0.3144, -0.1638,  0.0070, -0.3749,
        -0.1362, -0.0282, -0.2794,  0.1125,  0.3155, -0.0216,  0.3642,  0.0322,
        -0.0636, -0.1054, -0.4663, -0.2764,  0.3828,  0.2129,  0.4273, -0.4804,
         0.0863, -0.0198,  0.1996,  0.0611,  0.1697,  0.3256, -0.0072,  0.1485,
         0.1177,  0.0783,  0.2742,  0.1788, -0.4421, -0.1873,  0.0389, -0.4962,
        -0.1433, -0.0109, -0.3560, -0.3070,  0.0098, -0.2011, -0.3037,  0.3213,
        -0.3695,  0.0084, -0.3305,  0.0211,  0.0382, -0.3180,  0.3647,  0.4145,
         0.2649,  0.2046, -0.2747, -0.2274,  0.0073,  0.0336, -0.1146,  0.1170,
         0.1001,  0.3465,  0.2782, -0.0322,  0.0253,  0.0297,  0.4198, -0.5326,
         0.3701, -0.4183, -0.4745,  0.1805,  0.0984, -0.2897, -0.3507,  0.4974,
         0.3743,  0.1803,  0.1201, -0.2476,  0.0435, -0.4809,  0.4869, -0.2047,
        -0.0063,  0.2796,  0.1696, -0.0494, -0.1329, -0.3734, -0.3793, -0.0420,
         0.1840, -0.0030, -0.0413, -0.12

In [11]:
thresholds = np.arange(0, 1, 0.001)
# thresholds = np.arange(0.4, 0.44, 0.0000001)

example_weight = [-0.0505,  0.2211]
racist_score = example_weight[1]
not_racist_score = example_weight[0]

softmax_hate_score = np.exp(racist_score) / (np.exp(racist_score) + np.exp(not_racist_score))
r = 0
nr = 0
for th in thresholds:
    if softmax_hate_score >= th:
        r += 1
    elif softmax_hate_score < th:
        nr = 0

# print(thresholds)
print(nr, r)

0 568
